In [10]:
!wget https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_single.jsonl
!wget https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_pair.jsonl

--2024-06-01 15:37:26--  https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_single.jsonl
Resolving huggingface.co (huggingface.co)... 18.165.183.94, 18.165.183.110, 18.165.183.98, ...
Connecting to huggingface.co (huggingface.co)|18.165.183.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/12/2b/122bd8e9eccbb3acc98acf73e0ecef3c96f24dcdb5f6639074ed304eb19f9cd4/76c55033c6b2b1cc3f62513458f84748a23352495fd42b1062a7401de5ff9bd9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27gpt-4_single.jsonl%3B+filename%3D%22gpt-4_single.jsonl%22%3B&Expires=1717508246&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzUwODI0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8xMi8yYi8xMjJiZDhlOWVjY2JiM2FjYzk4YWNmNzNlMGVjZWYzYzk2ZjI0ZGNkYjVmNjYzOTA3NGVkMzA0ZWIxOWY5Y2Q0Lzc2YzU1MDMzYzZiMmIxY2MzZjYyNTEzNDU4Zjg0NzQ4YTIzMzUyN

In [1]:
# !pip install -U plotly kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.8 MB/s eta 0:00:0000:0100:01


In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import os
os.chdir("data/mt_bench/model_judgment")


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
df_pair = get_model_df_pair()

In [2]:
df_pair

,qid,turn,result,category,model
0,81,1,loss,Writing,alpaca-13b
1,81,2,loss,Writing,alpaca-13b
2,82,1,loss,Writing,alpaca-13b
3,82,2,loss,Writing,alpaca-13b
4,83,1,loss,Writing,alpaca-13b
...,...,...,...,...,...
4870,121,1,win,Coding,phi-3
4871,122,1,tie,Coding,phi-3
4872,123,1,tie,Coding,phi-3
4873,124,1,tie,Coding,phi-3


In [3]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        # # pairwise result
        # res_pair = df_pair[(df_pair["category"]==cat) & (df_pair["model"]==model)]["result"].value_counts()
        # wincnt = res_pair["win"] if "win" in res_pair.index else 0
        # tiecnt = res_pair["tie"] if "tie" in res_pair.index else 0
        # winrate = wincnt/res_pair.sum()
        # winrate_adjusted = (wincnt + tiecnt)/res_pair.sum()
        # # print(winrate_adjusted)

        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})

['alpaca-13b' 'baize-v2-13b' 'chatglm-6b' 'claude-instant-v1' 'claude-v1'
 'dolly-v2-12b' 'falcon-40b-instruct' 'fastchat-t5-3b' 'gpt-3.5-turbo'
 'gpt-4' 'gpt4all-13b-snoozy' 'guanaco-33b' 'guanaco-65b'
 'h2ogpt-oasst-open-llama-13b' 'koala-13b' 'llama-13b' 'mpt-30b-chat'
 'mpt-30b-instruct' 'mpt-7b-chat' 'nous-hermes-13b'
 'oasst-sft-4-pythia-12b' 'oasst-sft-7-llama-30b' 'palm-2-chat-bison-001'
 'rwkv-4-raven-14b' 'stablelm-tuned-alpha-7b' 'tulu-30b' 'vicuna-13b-v1.3'
 'vicuna-33b-v1.3' 'vicuna-7b-v1.3' 'wizardlm-13b' 'wizardlm-30b'
 'Llama-2-7b-chat' 'Llama-2-13b-chat' 'Llama-2-70b-chat' 'phi-3-dpo']


In [8]:
target_models = ["phi-3", "Llama-2-7b-chat", "gpt-3.5-turbo", "phi-3-dpo", "gpt-4"]

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

rename_map = {"llama-13b": "LLaMA-13B",
              "alpaca-13b": "Alpaca-13B",
              "vicuna-33b-v1.3": "Vicuna-33B",
              "vicuna-13b-v1.3": "Vicuna-13B",
              "gpt-3.5-turbo": "GPT-3.5-turbo",
              "claude-v1": "Claude-v1",
              "gpt-4": "GPT-4"}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)


In [9]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'model=GPT-4<br>score=%{r}<br>category=%{theta}<extra></extra>',
              'legendgroup': 'GPT-4',
              'line': {'color': 'rgb(102, 197, 204)', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'GPT-4',
              'r': array([9.65 , 8.9  , 9.   , 6.8  , 8.55 , 9.375, 9.7  , 9.95 , 9.65 ]),
              'showlegend': True,
              'subplot': 'polar',
              'theta': array(['Writing', 'Roleplay', 'Reasoning', 'Math', 'Coding', 'Extraction',
                              'STEM', 'Humanities', 'Writing'], dtype=object),
              'type': 'scatterpolar'},
             {'hovertemplate': 'model=phi-3-dpo<br>score=%{r}<br>category=%{theta}<extra></extra>',
              'legendgroup': 'phi-3-dpo',
              'line': {'color': 'rgb(246, 207, 113)', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'phi-3-dpo',
              'r': array([9.6 , 8.55, 6.85, 6.15, 6.45, 8.25, 9.8 , 9.95, 9.6 ]),
              'showlegend': True,
              'subplot': 'polar',
              'theta': array(['Writing', 'Roleplay', 'Reasoning', 'Math', 'Coding', 'Extraction',
                              'STEM', 'Humanities', 'Writing'], dtype=object),
              'type': 'scatterpolar'},
             {'hovertemplate': 'model=GPT-3.5-turbo<br>score=%{r}<br>category=%{theta}<extra></extra>',
              'legendgroup': 'GPT-3.5-turbo',
              'line': {'color': 'rgb(248, 156, 116)', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'GPT-3.5-turbo',
              'r': array([9.2 , 8.4 , 5.65, 6.3 , 6.9 , 8.85, 8.7 , 9.55, 9.2 ]),
              'showlegend': True,
              'subplot': 'polar',
              'theta': array(['Writing', 'Roleplay', 'Reasoning', 'Math', 'Coding', 'Extraction',
                              'STEM', 'Humanities', 'Writing'], dtype=object),
              'type': 'scatterpolar'},
             {'hovertemplate': 'model=Llama-2-7b-chat<br>score=%{r}<br>category=%{theta}<extra></extra>',
              'legendgroup': 'Llama-2-7b-chat',
              'line': {'color': 'rgb(220, 176, 242)', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Llama-2-7b-chat',
              'r': array([8.9 , 7.7 , 4.25, 2.4 , 3.  , 6.5 , 8.65, 8.75, 8.9 ]),
              'showlegend': True,
              'subplot': 'polar',
              'theta': array(['Writing', 'Roleplay', 'Reasoning', 'Math', 'Coding', 'Extraction',
                              'STEM', 'Humanities', 'Writing'], dtype=object),
              'type': 'scatterpolar'}],
    'layout': {'legend': {'title': {'text': 'model'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'polar': {'angularaxis': {'categoryarray': [Writing, Roleplay,
                                                           Reasoning, Math, Coding,
                                                           Extraction, STEM,
                                                           Humanities],
                                         'categoryorder': 'array',
                                         'direction': 'clockwise',
                                         'rotation': 90},
                         'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]}},
               'template': '...'}
})